In [4]:
import numpy as np
import cvxpy as cp

vertex_names = {0: 'Origin', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'Destination'}

M = 10**10
c = np.array([[M, 40, 60, 50, M, M, M],
              [M, M, 10, M, 70, M, M],
              [M, M, M, 20, 55, 40, M],
              [M, M, M, M, M, 50, M],
              [M, M, M, M, M, 10, 60],
              [M, M, M, M, M, M, 80]])

# Dynamically initialize variables only for entries where c > M, i.e. edges that exist in the graph
x = {(i, j): cp.Variable(boolean=True) for i in range(c.shape[0]) for j in range(c.shape[1]) if c[i, j] < M}

objective = cp.Minimize(cp.sum([c[i, j] * x[i, j] for (i, j) in x]))

constraints = []

# Enforce exit from the origin
constraints.append(cp.sum([x[0, j] for j in range(c.shape[1]) if (0, j) in x]) == 1)

# Flow conservation at each node except origin and destination
for j in range(1, 6):
    inflow = cp.sum([x[i, j] for i in range(c.shape[0]) if (i, j) in x])
    outflow = cp.sum([x[j, k] for k in range(c.shape[1]) if (j, k) in x])
    constraints.append(inflow == outflow)

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GUROBI)

print(f'Problem status: {prob.status}')
print(f'Optimal value: {prob.value}')
print('Optimal solution:')
for (i, j), var in x.items():
    print(f'\t x{(i+1, j+1)} = {var.value}')

# Print optimal path
print('Optimal path:', end=' ')
current_node = 0  
while current_node != 6:  
    print(vertex_names[current_node], end=' -> ')
    next_node = next(j for j in range(c.shape[1]) if (current_node, j) in x and x[current_node, j].value == 1)
    current_node = next_node
print(vertex_names[current_node])


Problem status: optimal
Optimal value: 165.0
Optimal solution:
	 x(1, 2) = 1.0
	 x(1, 3) = 0.0
	 x(1, 4) = 0.0
	 x(2, 3) = 1.0
	 x(2, 5) = 0.0
	 x(3, 4) = 0.0
	 x(3, 5) = 1.0
	 x(3, 6) = 0.0
	 x(4, 6) = 0.0
	 x(5, 6) = 0.0
	 x(5, 7) = 1.0
	 x(6, 7) = 0.0
Optimal path: Origin -> A -> B -> D -> Destination


In [2]:
import numpy as np
import cvxpy as cp
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
import matplotlib

# Enable PGF backend for LaTeX-style output
matplotlib.use("pgf")
plt.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "font.family": "sans-serif",
    "text.usetex": True,
    "pgf.rcfonts": False,
})

# Define vertices and costs
vertex_names = {0: 'Origin', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'Dest.'}
M = 10**10
c = np.array([[M, 40, 60, 50, M, M, M],
              [M, M, 10, M, 70, M, M],
              [M, M, M, 20, 55, 40, M],
              [M, M, M, M, M, 50, M],
              [M, M, M, M, M, 10, 60],
              [M, M, M, M, M, M, 80]])

# Define the optimization problem
x = cp.Variable((6, 7), boolean=True)
objective = cp.Minimize(cp.sum(cp.multiply(c, x)))
constraints = [cp.sum(x[0, :]) == 1]  # Enforce exit from origin
constraints += [cp.sum(x[j, :]) - cp.sum(x[:, j]) == 0 for j in range(1, 6)]  # Conserve flow
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GUROBI)

# Create a NetworkX graph
G = nx.DiGraph()

# Add nodes
for i, name in vertex_names.items():
    G.add_node(i, label=name)

# Add edges with weights
for i in range(6):
    for j in range(7):
        if c[i, j] < M:  # Only add finite-cost edges
            G.add_edge(i, j, weight=c[i, j])

# Define smaller positions for plotting to make nodes appear larger in comparison
positions = {
    0: (0, 0), 1: (0.2, 0.3), 2: (0.2, -0.3), 
    3: (0.4, 0.3), 4: (0.6, 0), 5: (0.4, -0.3), 6: (0.8, 0)
}

# Styling parameters
node_style = dict(node_size=3000, node_color="white", edgecolors="black", font_size=16, font_weight="bold")
label_font = dict(font_size=14, font_weight="bold")

# Function to draw arrows with highlighting for selected edges
def draw_custom_arrow(start, end, color="black", width=2, mutation_scale=25):
    # Calculate direction vector and adjust the end point along this direction
    direction = np.array(end) - np.array(start)
    distance = np.sqrt(direction[0]**2 + direction[1]**2)
    norm_direction = direction / distance
    adjusted_end = np.array(end) - 0.05 * norm_direction  # Adjust end closer to start by 0.05 units

    arrow = FancyArrowPatch(
        start, adjusted_end, arrowstyle='-|>', color=color,
        mutation_scale=mutation_scale, lw=width
    )
    plt.gca().add_patch(arrow)

# Plot the original graph with all edges
plt.figure(figsize=(10, 8))
nx.draw(G, pos=positions, labels=vertex_names, with_labels=True, node_shape="o", **node_style)

for (i, j), label in nx.get_edge_attributes(G, "weight").items():
    start = np.array(positions[i])
    end = np.array(positions[j])
    draw_custom_arrow(start, end)

    # Place the label at two-thirds along the edge
    x_label = start[0] + 2/3 * (end[0] - start[0])
    y_label = start[1] + 2/3 * (end[1] - start[1])
    
    # Draw the label with a circular background
    plt.text(x_label, y_label, f"{label}", ha="center", va="center",
             bbox=dict(facecolor="white", edgecolor="black", boxstyle="circle,pad=0.3"),
             fontsize=14, weight="bold")

plt.savefig("original_graph_large_nodes.pgf", bbox_inches="tight")  # Save as PGF file
plt.show()

# Plot the solution graph with highlighted selected edges
plt.figure(figsize=(10, 8))
nx.draw(G, pos=positions, labels=vertex_names, with_labels=True, node_shape="o", **node_style)

for (i, j), label in nx.get_edge_attributes(G, "weight").items():
    start = np.array(positions[i])
    end = np.array(positions[j])
    
    # Check if the edge is part of the solution (x[i, j] = 1)
    if x.value[i, j] > 0.5:
        # Highlight selected edges with a distinct color and width
        draw_custom_arrow(start, end, color="blue", width=3)
    else:
        draw_custom_arrow(start, end)

    # Place the label at two-thirds along the edge
    x_label = start[0] + 2/3 * (end[0] - start[0])
    y_label = start[1] + 2/3 * (end[1] - start[1])
    
    # Draw the label with a circular background
    plt.text(x_label, y_label, f"{label}", ha="center", va="center",
             bbox=dict(facecolor="white", edgecolor="black", boxstyle="circle,pad=0.3"),
             fontsize=14, weight="bold")

plt.savefig("solution_graph_large_nodes.pgf", bbox_inches="tight")  # Save as PGF file
plt.show()


C:\Users\Yoni\AppData\Local\Temp\ipykernel_34396\3747694797.py:91: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\Yoni\AppData\Local\Temp\ipykernel_34396\3747694797.py:118: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [ ]:
import numpy as np
import cvxpy as cp

vertex_names = {0: 'Origin', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'Destination'}

M = -10**10
c = np.array([[M, 40, 60, 50, M, M, M],
              [M, M, 10, M, 70, M, M],
              [M, M, M, 20, 55, 40, M],
              [M, M, M, M, M, 50, M],
              [M, M, M, M, M, 10, 60],
              [M, M, M, M, M, M, 80]])

f = {(i, j): cp.Variable() for i in range(c.shape[0]) for j in range(c.shape[1]) if c[i, j] > M}

objective = cp.Maximize(cp.sum([f[0, j] for j in range(1, c.shape[1]) if (0, j) in f]))

constraints = []

# Flow conservation constraints for nodes (excluding source and sink)
for j in range(1, 6):
    inflow = cp.sum([f[i, j] for i in range(c.shape[0]) if (i, j) in f])
    outflow = cp.sum([f[j, k] for k in range(c.shape[1]) if (j, k) in f])
    constraints.append(inflow == outflow)

# Capacity constraints and non-negativity constraints
for (i, j) in f:
    constraints.append(f[i, j] <= c[i, j])  # Capacity constraint
    constraints.append(f[i, j] >= 0)        # Non-negativity constraint

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.GUROBI)

print(f'Problem status: {prob.status}')
print(f'Optimal value: {prob.value}')
print('Optimal solution:')
for (i, j), var in f.items():
    print(f'\t f{(i+1, j+1)} = {var.value}')

Problem status: optimal
Optimal value: 140.0
Optimal solution:
	 f(1, 2) = 30.0
	 f(1, 3) = 60.0
	 f(1, 4) = 50.0
	 f(2, 3) = 0.0
	 f(2, 5) = 30.0
	 f(3, 4) = 0.0
	 f(3, 5) = 30.0
	 f(3, 6) = 30.0
	 f(4, 6) = 50.0
	 f(5, 6) = 0.0
	 f(5, 7) = 60.0
	 f(6, 7) = 80.0
